# **Question and Answering App**

Langchain with RAG and BagelDB Vectore Store

Used Google VertexAI API for chat model

# Install Dependencies





In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub bs4 langchain-core
!pip install -qU langchain-google-vertexai

# Setup Environment Variables


In [ ]:
from google.colab import userdata
import os

HUGGINGFACEHUB_API_TOKEN = userdata.get('huggingface')
OPENAI_API_KEY = userdata.get('Openai')
GOOGLE_API_KEY = userdata.get('Googleapi')
LANGCHAIN_API_KEY = userdata.get('lanchain')
PROJECT_ID = userdata.get('projectid')

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# Install required packages

In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Setup Google Cloud AI platform to access VertexAI API enabled in Project

In [ ]:
!pip install google-cloud-aiplatform
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID)

In [ ]:
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(model="gemini-pro")

# Install BagelDB Vectore store

In [ ]:
from langchain_community.vectorstores import Bagel

In [ ]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=500)
splits = text_splitter.split_documents(docs)

# create cluster with docs
cluster = Bagel.from_documents(cluster_name="listed_docs", documents=splits)
retriever = cluster.as_retriever()

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Google authentication setup to access Google cloud account

In [ ]:
!pip install google-auth --upgrade
from google.colab import auth
auth.authenticate_user()


# Testing with RAG chain

In [ ]:
rag_chain.invoke("What is Task Decomposition?")

In [ ]:
# cleanup storage of vectore DB
#vectorstore.delete_collection()

# Implement QA with chat history

Define sub chain for retrive hostorical messages and take latest question from user.

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

### Contextualize question ###
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

### Answer question ###
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

## Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

In [ ]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

# Steaming the QA history chain

In [ ]:
from operator import itemgetter
from langchain_core.tracers.log_stream import LogStreamCallbackHandler
import nest_asyncio
from langchain_core.messages import HumanMessage


nest_asyncio.apply()
contextualize_q_chain = (contextualize_q_prompt | llm | StrOutputParser()).with_config(
    tags=["contextualize_q_chain"]
)

def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input["input"]



chat_history = []

question = "What is Task Decomposition?"
ai_msg = conversational_rag_chain.invoke({"input": question, "chat_history": chat_history},config={"configurable": {"session_id": "abc123"}},)
chat_history.extend([HumanMessage(content=question), ai_msg])

second_question = "What are common ways of doing it?"
ct = 0
async for jsonpatch_op in conversational_rag_chain.astream_log(
    {"input": second_question, "chat_history": chat_history},
    include_tags=["contextualize_q_chain"],config={"configurable": {"session_id": "abc123"}},
):
    print(jsonpatch_op)
    print("\n" + "-" * 30 + "\n")
    ct += 1
    if ct > 20:
        break